In [1]:
f = open("C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/HaeinAPIKey.txt", "r")
API_Key = f.readline()
API_Key = API_Key.strip("\n")
decoded_API_Key =f.readline()
f.close()

In [2]:
API_Key = decoded_API_Key

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
def sigunguDF(data):
    item_list = data.find_all('item')
    data_list = []
    for item in item_list:
        if(item.find("kaptName")!=None):
            Complex_Name = item.find("kaptName").get_text()
        else:
            Complex_Name = item.find("kaptname").get_text()
        
        if(item.find("kaptCode")!=None):
            kaptCode = item.find("kaptCode").get_text()
        else:
            kaptCode = item.find("kaptcode").get_text()

        if(item.find("bjdCode")!=None):
            bjdCode = item.find("bjdCode").get_text()
        else:
            bjdCode = item.find("bjdcode").get_text()

        data_dict = {
                "Complex_Name": Complex_Name,
                "kaptCode": kaptCode,
                "bjdCode": bjdCode,
            }
        data_list.append(data_dict)

    df = pd.DataFrame(data_list)
    return df

In [5]:
def getSigunguAptList(API_Key, code):
    """
    Input sigunguCode to get list of Apartment complexes and their codes
    code -- sigunguCode input; Code of the region given by the government
    """
    url = "http://apis.data.go.kr/1613000/AptListService2/getSigunguAptList"
    params ={'serviceKey' : API_Key , 
             'sigunguCode' : code, 
             'pageNo': 1,
             }

    response = requests.get(url, params=params)
    first_data = BeautifulSoup(response.text, "xml")

    df_init = sigunguDF(first_data)
    if(first_data.find('totalCount')!=None):
        total = int(first_data.find('totalCount').get_text())
    else:
        total = int(first_data.find('totalcount').get_text())

    totPage = (total // 10) + 2
    
    for page in range(2,totPage):    
        url = "http://apis.data.go.kr/1613000/AptListService2/getSigunguAptList"
        params ={'serviceKey' : API_Key , 
                'sigunguCode' : code, 
                'pageNo': page}

        response = requests.get(url, params=params)
        data = BeautifulSoup(response.text, "xml")
        df = sigunguDF(data)
        df_init = pd.concat([df_init, df], ignore_index=True)
        
    return df_init

In [6]:
def AddInfo(df):
    AptList = []
    for Apt in df['kaptCode']:
        url = "http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo"
        params ={'serviceKey' : API_Key , 
            'kaptCode' : Apt, 
            }
        response = requests.get(url, params=params)
        data = BeautifulSoup(response.text, "xml")
        if(data.find('kaptdaCnt') != None):
            Household_Num = data.find('kaptdaCnt').get_text()
        elif(data.find('kaptdacnt')!=None): 
            Household_Num = data.find('kaptdacnt').get_text()
        else: Household_Num = np.nan

        if(data.find('kaptUsedate')!=None):
            Begin_Date = data.find('kaptUsedate').get_text()
        elif(data.find('kaptusedate')!=None):
            Begin_Date = data.find('kaptusedate').get_text()
        else: Begin_Date = np.nan

        if(data.find('codeHeatNm')!=None):
            Heating_Method = data.find('codeHeatNm').get_text()
        elif(data.find('codeheatnm')!=None): 
            Heating_Method = data.find('codeheatnm').get_text()
        else:
            Heating_Method = np.nan

        if(data.find('codeAptNm')!=None):
            House_Type = data.find('codeAptNm').get_text()
        elif(data.find('codeaptnm')!=None): 
            House_Type = data.find('codeaptnm').get_text()
        else:
            House_Type = np.nan

        data_dict = {
        "kaptCode": Apt,
        "Household_Num":Household_Num,
        "Begin_Date":Begin_Date,
        "Heating_Method":Heating_Method,
        "House_Type":House_Type,
        }
        AptList.append(data_dict)

    df_2 = pd.DataFrame(AptList)
    df = df.merge(df_2, on= "kaptCode")
    return df


In [7]:
def cityIn(sigunguCode, API_Key, GuName):
    data =  getSigunguAptList(API_Key, sigunguCode)
    data = AddInfo(data)
    display(data)
    data.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Energy/AptList/{GuName}.csv")

In [59]:
cityIn(11740, API_Key, "Gangdong_Gu")

,Complex_Name,kaptCode,bjdCode,Household_Num,Begin_Date,Heating_Method,House_Type
0,명일신동아,A13407204,1174010100,570,19860208,중앙난방,아파트
1,명일삼익그린2차,A13407104,1174010100,2400,19831217,중앙난방,아파트
2,고덕현대아파트,A13478601,1174010100,524,19860207,중앙난방,아파트
3,"명일삼익가든1,2차",A13407002,1174010100,768,19841016,개별난방,아파트
4,고덕주공9단지,A13482706,1174010100,1320,19851125,중앙난방,아파트
...,...,...,...,...,...,...,...
114,강동리버스트6단지,A10024422,1174011000,1244,20210223,지역난방,아파트
115,강일푸르내,A10024487,1174011000,119,20200123,개별난방,아파트
116,강동리버스트4단지,A10024468,1174011000,1239,20200828,지역난방,아파트
117,강동리버스트 8단지,A10024440,1174011000,946,20210210,지역난방,아파트
